In [1]:
# Importing necessary packages
import requests
import json
import urllib
import shapely.geometry
import pyproj
import math
import folium
import pandas as pd
import pickle
import numpy as np
from sklearn.cluster import KMeans
from folium import plugins
from folium.plugins import HeatMap

In [2]:
# Api_key was removed for publishing purposes
google_api_key = ''

## Obtaining initial data

In [3]:
#Let's find a center of the Yekateriburg using Google maps API
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location']
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Ulitsa Malysheva, Yekaterinburg, Russia'
Yekaterinburg_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, Yekaterinburg_center))

Coordinate of Ulitsa Malysheva, Yekaterinburg, Russia: [56.836758, 60.6257876]


In [4]:
#For further analysis Latitude and Longtitude should be transformed into UTM format
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Yekaterinburg center longitude={}, latitude={}'.format(Yekaterinburg_center[1], Yekaterinburg_center[0]))
x, y = lonlat_to_xy(Yekaterinburg_center[1], Yekaterinburg_center[0])
print('Yekaterinburg center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Yekaterinburg center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Yekaterinburg center longitude=60.6257876, latitude=56.836758
Yekaterinburg center UTM X=3139242.9260965036, Y=7258937.6829704475
Yekaterinburg center longitude=60.62578759999999, latitude=56.836758


<ipython-input-4-5708a64978fb>:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)


In [5]:
#Generate neighbours around the center with radius = 800m

Yekaterinburg_center_x, Yekaterinburg_center_y = lonlat_to_xy(Yekaterinburg_center[1], Yekaterinburg_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = Yekaterinburg_center_x - 6000
x_step = 1600
y_min = Yekaterinburg_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 1600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 800 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(Yekaterinburg_center_x, Yekaterinburg_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

<ipython-input-4-5708a64978fb>:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This funct

<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This func

48 candidate neighborhood centers generated.


<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)


In [6]:
#Place each neighbourhoods on the map around the city's center

map_Yekaterinburg = folium.Map(location=Yekaterinburg_center, zoom_start=13)
x_ = 0
folium.Marker(Yekaterinburg_center, popup='Ulitsa Malysheva').add_to(map_Yekaterinburg)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=735, color='blue', fill=False).add_to(map_Yekaterinburg)
map_Yekaterinburg

In [7]:
#Using Google API addresses of neighbourhoods will be collected

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, Yekaterinburg_center[0], Yekaterinburg_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(Yekaterinburg_center[0], Yekaterinburg_center[1], addr))

Reverse geocoding check
-----------------------
Address of [56.836758, 60.6257876] is: Ulitsa Malysheva, 75, Yekaterinburg, Sverdlovskaya oblast', Russia, 620075


In [8]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Russia', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


## Collecting addresses of target regions

In [9]:
df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Ulitsa Repina, 64, Yekaterinburg, Sverdlovskay...",56.821315,60.546160,3.136443e+06,7.254088e+06,5600.000000
1,"Ulitsa Volgogradskaya, 182А, Yekaterinburg, Sv...",56.812725,60.564509,3.138043e+06,7.254088e+06,4995.998399
2,"Ulitsa Raskovoy, 56, Yekaterinburg, Sverdlovsk...",56.804133,60.582848,3.139643e+06,7.254088e+06,4866.210024
3,"Ulitsa 8 Marta, 204Ж, Yekaterinburg, Sverdlovs...",56.795539,60.601177,3.141243e+06,7.254088e+06,5245.950819
4,"Zavodskaya Ulitsa, 32 корпус 3, Yekaterinburg,...",56.834326,60.550550,3.135643e+06,7.255474e+06,4995.998399
5,"Спецавтобаза, Posadskaya Ulitsa, 3, Yekaterinb...",56.825734,60.568904,3.137243e+06,7.255474e+06,4000.000000
6,"Moskovskaya St, 88, Yekaterinburg, Sverdlovska...",56.817140,60.587249,3.138843e+06,7.255474e+06,3487.119155
7,"Ulitsa Shchorsa, 110, Yekaterinburg, Sverdlovs...",56.808544,60.605582,3.140443e+06,7.255474e+06,3666.060556
8,"Chaykovskogo St, 86Б, Yekaterinburg, Sverdlovs...",56.799947,60.623906,3.142043e+06,7.255474e+06,4454.211490
9,"Samoletnaya Ulitsa, 1, Yekaterinburg, Sverdlov...",56.791347,60.642220,3.143643e+06,7.255474e+06,5600.000000


In [10]:
df_locations.to_pickle('./locations.pkl')    

In [11]:
foursquare_client_id = '' #removed for publishing purposes
foursquare_client_secret = '' #removed for publishing purposes

## Define venues in target neighbourhoods using Foursquare

In [12]:
gym_category = '' #removed for publishing purposes

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20210301'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [13]:
def get_gyms(lats, lons):
    gyms = {}
    location_gyms = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any gym (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, gym_category, foursquare_client_id, foursquare_client_secret, radius=850, limit=100)
        area_gyms = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
            gym = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, x, y)
            if venue_distance<=800:
                area_gyms.append(gym)
            gyms[venue_id] = gym
        location_gyms.append(area_gyms)
        print(' .', end='')
    print(' done.')
    return gyms, location_gyms

# Try to load from local file system in case we did this before
gyms = {}
location_gyms = []
loaded = False
try:
    with open('gyms_350.pkl', 'rb') as f:
        gyms = pickle.load(f)
    with open('location_gyms_350.pkl', 'rb') as f:
        location_gyms = pickle.load(f)
    print('Gym data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    gyms, location_gyms = get_gyms(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('gyms_350.pkl', 'wb') as f:
        pickle.dump(gyms, f)
    with open('location_gyms_350.pkl', 'wb') as f:
        pickle.dump(location_gyms, f)

Gym data loaded.


In [14]:
print('Total number of gyms:', len(gyms))
print('Average number of gyms in neighborhood:', np.array([len(r) for r in location_gyms]).mean())

Total number of gyms: 187
Average number of gyms in neighborhood: 4.0625


## Placing found gyms on the map

In [15]:
map_Yekaterinburg = folium.Map(location=Yekaterinburg_center, zoom_start=13)
folium.Marker(Yekaterinburg_center, popup='Ulitsa Malysheva').add_to(map_Yekaterinburg)
for g in gyms.values():
    lat = g[2]; lon = g[3]
    folium.CircleMarker([lat, lon], radius=3, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Yekaterinburg)
map_Yekaterinburg

## Adding features for further analysis:
- Number of gyms in the area
- Distance from the center to the nearest gyms

In [16]:
location_gyms_count = [len(g) for g in location_gyms]
df_locations['Gyms in area'] = location_gyms_count
print('Average number of gyms in every area with radius=800m:', np.array(location_gyms_count).mean())
df_locations.head(10)

Average number of gyms in every area with radius=800m: 4.0625


,Address,Latitude,Longitude,X,Y,Distance from center,Gyms in area
0,"Ulitsa Repina, 64, Yekaterinburg, Sverdlovskay...",56.821315,60.546160,3.136443e+06,7.254088e+06,5600.000000,3
1,"Ulitsa Volgogradskaya, 182А, Yekaterinburg, Sv...",56.812725,60.564509,3.138043e+06,7.254088e+06,4995.998399,3
2,"Ulitsa Raskovoy, 56, Yekaterinburg, Sverdlovsk...",56.804133,60.582848,3.139643e+06,7.254088e+06,4866.210024,1
3,"Ulitsa 8 Marta, 204Ж, Yekaterinburg, Sverdlovs...",56.795539,60.601177,3.141243e+06,7.254088e+06,5245.950819,4
4,"Zavodskaya Ulitsa, 32 корпус 3, Yekaterinburg,...",56.834326,60.550550,3.135643e+06,7.255474e+06,4995.998399,7
5,"Спецавтобаза, Posadskaya Ulitsa, 3, Yekaterinb...",56.825734,60.568904,3.137243e+06,7.255474e+06,4000.000000,4
6,"Moskovskaya St, 88, Yekaterinburg, Sverdlovska...",56.817140,60.587249,3.138843e+06,7.255474e+06,3487.119155,9
7,"Ulitsa Shchorsa, 110, Yekaterinburg, Sverdlovs...",56.808544,60.605582,3.140443e+06,7.255474e+06,3666.060556,12
8,"Chaykovskogo St, 86Б, Yekaterinburg, Sverdlovs...",56.799947,60.623906,3.142043e+06,7.255474e+06,4454.211490,3
9,"Samoletnaya Ulitsa, 1, Yekaterinburg, Sverdlov...",56.791347,60.642220,3.143643e+06,7.255474e+06,5600.000000,4


In [17]:
distances_to_gyms = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for g in gyms.values():
        g_x = g[6]
        g_y = g[7]
        d = calc_xy_distance(area_x, area_y, g_x, g_y)
        if d<min_distance:
            min_distance = d
    distances_to_gyms.append(min_distance)

df_locations['Distance to Gym'] = distances_to_gyms

In [18]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Gyms in area,Distance to Gym
0,"Ulitsa Repina, 64, Yekaterinburg, Sverdlovskay...",56.821315,60.546160,3.136443e+06,7.254088e+06,5600.000000,3,107.213547
1,"Ulitsa Volgogradskaya, 182А, Yekaterinburg, Sv...",56.812725,60.564509,3.138043e+06,7.254088e+06,4995.998399,3,256.644803
2,"Ulitsa Raskovoy, 56, Yekaterinburg, Sverdlovsk...",56.804133,60.582848,3.139643e+06,7.254088e+06,4866.210024,1,692.632968
3,"Ulitsa 8 Marta, 204Ж, Yekaterinburg, Sverdlovs...",56.795539,60.601177,3.141243e+06,7.254088e+06,5245.950819,4,368.515802
4,"Zavodskaya Ulitsa, 32 корпус 3, Yekaterinburg,...",56.834326,60.550550,3.135643e+06,7.255474e+06,4995.998399,7,393.633315
5,"Спецавтобаза, Posadskaya Ulitsa, 3, Yekaterinb...",56.825734,60.568904,3.137243e+06,7.255474e+06,4000.000000,4,244.397575
6,"Moskovskaya St, 88, Yekaterinburg, Sverdlovska...",56.817140,60.587249,3.138843e+06,7.255474e+06,3487.119155,9,461.535233
7,"Ulitsa Shchorsa, 110, Yekaterinburg, Sverdlovs...",56.808544,60.605582,3.140443e+06,7.255474e+06,3666.060556,12,105.700275
8,"Chaykovskogo St, 86Б, Yekaterinburg, Sverdlovs...",56.799947,60.623906,3.142043e+06,7.255474e+06,4454.211490,3,539.924280
9,"Samoletnaya Ulitsa, 1, Yekaterinburg, Sverdlov...",56.791347,60.642220,3.143643e+06,7.255474e+06,5600.000000,4,291.352229


In [19]:
print('Average distance to closest gym from each area center:', df_locations['Distance to Gym'].mean())

Average distance to closest gym from each area center: 553.0940512248609


In [20]:
gyms_latlons = [[g[2], g[3]] for g in gyms.values()]

## Let's add heat map to investigate the density of observed location

In [21]:
map_Yekaterinburg = folium.Map(location=Yekaterinburg_center, zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(map_Yekaterinburg) #cartodbpositron cartodbdark_matter
HeatMap(gyms_latlons).add_to(map_Yekaterinburg)
folium.Marker(Yekaterinburg_center).add_to(map_Yekaterinburg)
folium.Circle(Yekaterinburg_center, radius=2000, fill=False, color='white').add_to(map_Yekaterinburg)
folium.Circle(Yekaterinburg_center, radius=4000, fill=False, color='white').add_to(map_Yekaterinburg)
folium.Circle(Yekaterinburg_center, radius=6000, fill=False, color='white').add_to(map_Yekaterinburg)
map_Yekaterinburg

In [22]:
good_gyms_count = np.array((df_locations['Gyms in area']<=3))
print('Locations with no more than two gyms nearby:', good_gyms_count.sum())

good_gyms_distance = np.array(df_locations['Distance to Gym']>=600)
print('Locations with no gyms within 600m:', good_gyms_distance.sum())

good_locations = np.logical_and(good_gyms_count, good_gyms_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_locations[good_locations]

Locations with no more than two gyms nearby: 27
Locations with no gyms within 600m: 20
Locations with both conditions met: 19


## I found 19 possible areas where gym can be opened

In [23]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_Yekaterinburg = folium.Map(location=Yekaterinburg_center, zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(map_Yekaterinburg)
HeatMap(gyms_latlons).add_to(map_Yekaterinburg)

folium.Marker(Yekaterinburg_center).add_to(map_Yekaterinburg)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Yekaterinburg) 
map_Yekaterinburg

In [24]:
#Marking regions where possible gym can be opened
x_ = 0
map_Yekaterinburg = folium.Map(location=Yekaterinburg_center, zoom_start=12)
HeatMap(good_locations, radius=25).add_to(map_Yekaterinburg)
folium.Marker(Yekaterinburg_center).add_to(map_Yekaterinburg)

for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Yekaterinburg)
    folium.Marker([lat, lon], popup=str(str(df_good_locations.index[x_]) + ' '+ str(good_latitudes[x_])+' '+ str(good_longitudes[x_]))).add_to(map_Yekaterinburg)
    x_ = x_ + 1
map_Yekaterinburg

### Some of found areas are placed in irrelevant zones (lake and manufacturing zone). Let's drop them

In [25]:
df_good_locations.drop([15, 23, 29, 30, 31, 37, 38, 44, 47], errors = 'ignore')
df_good_locations

,Address,Latitude,Longitude,X,Y,Distance from center,Gyms in area,Distance to Gym
2,"Ulitsa Raskovoy, 56, Yekaterinburg, Sverdlovsk...",56.804133,60.582848,3.139643e+06,7.254088e+06,4866.210024,1,692.632968
15,"Bazovyy Pereulok, 56, Yekaterinburg, Sverdlovs...",56.804349,60.646640,3.142843e+06,7.256859e+06,4156.921938,3,761.079708
16,"Unnamed Road, Yekaterinburg, Sverdlovskaya obl...",56.795746,60.664948,3.144443e+06,7.256859e+06,5600.000000,0,1331.957212
17,"Ulitsa Ol'khovskaya, 52, Yekaterinburg, Sverdl...",56.860349,60.559347,3.134043e+06,7.258245e+06,5245.950819,2,610.635855
22,"Сибирский тракт, 12/31, Yekaterinburg, Sverdlo...",56.817351,60.651065,3.142043e+06,7.258245e+06,2884.441020,1,752.102162
23,"Unnamed Road, Yekaterinburg, Sverdlovskaya obl...",56.808746,60.669378,3.143643e+06,7.258245e+06,4454.211490,0,1043.830911
28,"Vishnevaya Ulitsa, 42, Yekaterinburg, Sverdlov...",56.830354,60.655496,3.141243e+06,7.259631e+06,2116.601049,3,627.959572
29,"Sinie Kamni, Yekaterinburg, Sverdlovskaya obla...",56.821746,60.673814,3.142843e+06,7.259631e+06,3666.060556,2,680.670412
30,"Сибирский Тракт 8 Км, 3, Yekaterinburg, Sverdl...",56.813138,60.692122,3.144443e+06,7.259631e+06,5245.950819,0,1826.812192
31,"Unnamed Road, Yekaterinburg, Sverdlovskaya obl...",56.877773,60.586538,3.134043e+06,7.261016e+06,5600.000000,1,819.611223


In [32]:
#using K means, let's define 8 regions to find a starting point which will help to find a proper place for gyms 
new_good_latitudes = df_good_locations['Latitude']
new_good_longitudes = df_good_locations['Longitude']

number_of_clusters = 8

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_Yekaterinburg = folium.Map(location=Yekaterinburg_center, zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(map_Yekaterinburg)
HeatMap(gyms_latlons).add_to(map_Yekaterinburg)

folium.Marker(Yekaterinburg_center).add_to(map_Yekaterinburg)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=1500, color='green', fill=True, fill_opacity=0.25).add_to(map_Yekaterinburg) 
for lat, lon in zip(new_good_latitudes, new_good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Yekaterinburg)

map_Yekaterinburg

<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-4-5708a64978fb>:10: DeprecationWarning: This func

In [27]:
candidate_area_addresses = []

for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon).replace(', Russia', '').replace(', Yekaterinburg, Sverdlovskaya oblast', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, Yekaterinburg_center_x, Yekaterinburg_center_y)


<ipython-input-4-5708a64978fb>:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
<ipython-input-4-5708a64978fb>:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
<ipython-input-4-5708a64978fb>:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
<ipython-input-4-5708a64978fb>:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
<ipython-input-4-5708a64978fb>:4: DeprecationWarning: This function 

## Finally I found 8 locations. The nearest territory should be investigated to find a proper place for a new gym.

In [28]:
map_Yekaterinburg = folium.Map(location=Yekaterinburg_center, zoom_start=12)
folium.Circle(Yekaterinburg_center, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_Yekaterinburg)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]]).add_to(map_Yekaterinburg) 
for lat, lon in zip(new_good_latitudes, new_good_longitudes):
    folium.Circle([lat, lon], radius=800, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.20).add_to(map_Yekaterinburg)
map_Yekaterinburg

In [29]:
candidate_area_addresses

["Sibirskiy Trakt, 14', 620100",
 "Unnamed Road', 620017",
 "Ulitsa Otdykha, 107', 620033",
 "Ulitsa Raskovoy, 56', 620146",
 "Ulitsa Ol'khovskaya, 52', 620141",
 "улица Просторная, 148', 620089",
 "Ulitsa Frontovykh Brigad, 18 корпус 2', 620017",
 "Ulitsa Istochnaya, 2а', 620072"]